In [72]:
import pandas as pd
import pyodbc
import Utils

In [73]:
aencConnection = pyodbc.connect("Driver={SQL Server};"
                                "Server=DESKTOP-8INVJ1O\SQLEXPRESS;"
                                "Database=AENC;"
                                "trusted_connection=yes")

In [74]:
product = pd.read_sql("SELECT * FROM product", aencConnection)
customer  = pd.read_sql("SELECT * FROM customer", aencConnection)
employee  = pd.read_sql("SELECT * FROM employee", aencConnection)
state  = pd.read_sql("SELECT * FROM state", aencConnection)
salesOrder  = pd.read_sql("SELECT * FROM sales_order", aencConnection)
salesOrderItem  = pd.read_sql("SELECT * FROM sales_order_item", aencConnection)

C:\Users\Max\AppData\Local\Temp\ipykernel_31972\2065893459.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product = pd.read_sql("SELECT * FROM product", aencConnection)
C:\Users\Max\AppData\Local\Temp\ipykernel_31972\2065893459.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer  = pd.read_sql("SELECT * FROM customer", aencConnection)
C:\Users\Max\AppData\Local\Temp\ipykernel_31972\2065893459.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employee  = pd.read_sql("SELECT * FROM employee", aencConnection)
C

# Product
add unit price from product to order because new product dataframe doesn't need this one but the merged fact table / star scheme does need it

In [75]:
# rename initial because id name doesnt match properly
selectedColumns = ['id','description' ,'name', 'Category', 'color', 'quantity', 'unit_price']
product = product[selectedColumns]
newColumnNames = ['prod_id','description' ,'name', 'Category', 'color', 'quantity', 'unit_price']
product.columns = newColumnNames

In [76]:
salesOrderItem = pd.merge(salesOrderItem, product[['prod_id', 'unit_price']],on='prod_id', how='left')

In [77]:
selectedColumns = ['prod_id','description' ,'name', 'Category', 'color', 'quantity']
productData = product[selectedColumns]

In [78]:
newColumnNames = ['PRODUCT_id' , 'PRODUCT_name','PRODUCT_sub_category', 'PRODUCT_category', 'PRODUCT_color', 'PRODUCT_storage_quantity']
productData.columns = newColumnNames

# Customer

In [79]:
customerData = pd.merge(customer, state, left_on='state', right_on='state_id' , how='inner')

In [80]:
selectedColumns = ['id', 'address', 'city', 'state_name', 'region', 'country', 'company_name']
customerData = customerData[selectedColumns]

In [81]:
newColumnNames = ['CUSTOMER_id', 'CUSTOMER_address', 'CUSTOMER_city', 'CUSTOMER_state', 'CUSTOMER_region', 'CUSTOMER_country', 'CUSTOMER_company_name']
customerData.columns = newColumnNames

# Employee

In [82]:
selectedColumns = ['emp_id', 'emp_fname', 'emp_lname', 'city']
employeeDate = employee[selectedColumns]

In [83]:
newColumnNames = ['EMPLOYEE_id','EMPLOYEE_first_name','EMPLOYEE_last_name','EMPLOYEE_city']
employeeDate.columns = newColumnNames

# Day

In [84]:
newFrames = Utils.splitFrames(salesOrder, 'order_date', [])
salesOrder = newFrames[0]
dateData = newFrames[1]

In [85]:
dateFormat = '%d-%b-%Y %H:%M:%S %p'
DAY_date = Utils.getDayDate(salesOrder, 'order_date', dateFormat)

# Order Detail

In [86]:
orderDetailJoin = pd.merge(salesOrder, salesOrderItem, on='id')

In [88]:
selectedColumns = ['id','cust_id', 'order_date', 'sales_rep', 'prod_id', 'unit_price', 'quantity']
orderDetailData = orderDetailJoin[selectedColumns]

In [89]:
newColumnNames = ['ORDER_DETAIL_NUMBER', 'CUSTOMER_id', 'DAY_date', 'EMPLOYEE_id', 'PRODUCT_id', 'ORDER_DETAIL_unit_price', 'ORDER_DETAIL_order_quantity']
orderDetailData.columns = newColumnNames